# IRIS EmbeddedPython

In [3]:
import iris

#Pandas dataframes
import pandas as pd
pd.options.display.max_rows = 9


In [3]:
print('\nInterSystems IRIS version:')
irisVersion = iris.cls('%SYSTEM.Version').GetVersion()
print(irisVersion)



InterSystems IRIS version:
IRIS for UNIX (Ubuntu Server LTS for x86-64 Containers) 2021.2 (Build 651U) Mon Jan 31 2022 18:07:01 EST


# SQL
iris.sql.exec('select * from ...')

In [4]:
statement = iris.sql.exec('SELECT * FROM %SYS.ProcessQuery')
df = statement.dataframe()
jsn = df.to_json
print(jsn)
#df
df.dtypes
print(df.columns)
print(df.size)


<bound method NDFrame.to_json of       id namespace          routine  linesexecuted  globalreferences state  \
0    394                    CONTROL              0                 0   RUN   
1    435                     WRTDMN            285              1328  RUNW   
2    436                     GARCOL              0                 0  RUNW   
3    437                     JRNDMN           1595               347  RUNW   
..   ...       ...              ...            ...               ...   ...   
31  1677      %SYS  %SYS.cspServer2           9014               290  READ   
32   665      %SYS  %SYS.cspServer2         263992              8755  READ   
33   666      %SYS  %SYS.cspServer2          20895               669  READ   
34  1678      %SYS  %SYS.cspServer2           9303               301  READ   

   pidexternal   username clientipaddress clientnodename  ... globalblocks  \
0          394                                            ...            0   
1          435                

# SQL

In [13]:

statement = iris.sql.exec('SELECT * FROM Titanic_Table.Passenger')
df = statement.dataframe()
df


,id,survived,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.2500,,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.9250,,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1000,C123,S
...,...,...,...,...,...,...,...,...,...,...,...,...
887,888,1,1,"Graham, Miss. Margaret Edith",female,19,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,0,1,2,W./C. 6607,23.4500,,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26,0,0,111369,30.0000,C148,C
890,891,0,3,"Dooley, Mr. Patrick",male,32,0,0,370376,7.7500,,Q
